# 数据处理

数据处理工作包括处理重复值、缺失值和异常值，生成新的列或者行等。

In [1]:
import os
import urllib.request
import zipfile
import pandas as pd

folder_path = os.path.join(os.getcwd(), "../data/pwt")
download_url = "https://www.rug.nl/ggdc/docs/pwt70_06032011version.zip"
file_name = download_url.split("/")[-1]
if not os.path.exists(folder_path):
    # 创建文件夹
    os.makedirs(folder_path)
    print(f"文件夹不存在，已创建。")

    zip_file_path = os.path.join(folder_path, file_name)

    urllib.request.urlretrieve(download_url, zip_file_path)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)
    print("数据已下载并解压缩。")
else:
    print(f"文件夹已存在，无需操作。")

df = pd.read_csv(os.path.join(folder_path, "pwt70_w_country_names.csv"))

文件夹已存在，无需操作。


## 处理重复值

检测数据集的记录是否存在重复，可以使用 `.duplicated()` 函数进行验证，但是该函数返回的是数据集每一行的检测结果，即 n 行数据会返回 n 个布尔值。为了能够得到最直接的结果，可以使用 `any` 函数。该函数表示的是在多个条件判断中，只要有一个条件为 True，则 `any` 返回的结果为 True。

In [2]:
any(df.duplicated())

False

如果有重复项，可以通过 `.drop_duplicated()` 删除。该函数有 `inplace` 参数，设置为 True 表示直接在原始数据集上做操作：`df.drop_duplicated(inplace = True)`。

## 处理缺失值

缺失值是指数据集中的某些观测存在遗漏。数据缺失可能有两方面原因：一方面是人为原因，如记录过程中的遗漏、个人隐私而不愿意透露等；另一方面是机器或设备的故障所导致。

可以使用 `.isna()` 或 `.isnull()` 函数查看缺失值。这两个函数返回 `DataFrame` 每个元素是否有缺失值，如果有缺失值，比如 N/A 或`null`，则返回 True；否则返回 False。

例如：通过 `.sum()` 查看每一列的空缺值计数。

In [3]:
df.isnull().sum()

country              0
isocode              0
year                 0
POP                  2
XRAT              1237
Currency_Unit     1237
ppp               2655
tcgdp             2655
cgdp              2655
cgdp2             2655
cda2              2655
cc                2655
cg                2655
ci                2655
p                 2655
p2                2655
pc                2655
pg                2655
pi                2655
openc             2655
cgnp              3095
y                 2655
y2                2655
rgdpl             2675
rgdpl2            2675
rgdpch            2675
kc                2675
kg                2675
ki                2675
openk             2675
rgdpeqa           2845
rgdpwok           3223
rgdpl2wok         3223
rgdpl2pe         10555
rgdpl2te          6001
rgdpl2th          9126
rgdptt            2655
dtype: int64

一般而言，遇到缺失值时，可以采用三种方法处理，分别是删除法、替换法或插补法。

删除法是指当缺失的观测比例非常低（如 5% 以内），直接删除存在缺失的观测，或者当某些变量的缺失比例非常高时（如 85% 以上），直接删除这些缺失的变量。
  
可以使用 `.dropna()` 函数删除有缺失值的行或列。具体形式：`df.dropna(axis=0, how='any', inplace=False)`。

这个函数有参数 `axis`，`axis` 用来指定要删除的轴。`axis=0` 表示删除行（默认），`axis=1` 表示删除列。`how` 用来指定删除的条件。`how='any'` 表示删除包含任何缺失值的行（默认），`how='all'` 表示只删除所有值都是缺失值的行。`inplace` 用于指定是否在原始 `DataFrame` 上进行修改，默认为 False，表示不修改原始 `DataFrame`，而是返回一个新的 `DataFrame`。

例如，删除包含任何缺失值的行。

In [4]:
ddf = df.dropna()

`.fillna()` 函数填补数据框中的缺失值。例如，将 `DataFrame` 中的缺失值用 0 填充。

In [5]:
fill_zero_df = df.fillna(0)

对于时间类型的数据集，可以用 `.fillna()` 的 `method` 参数，可以接受 `ffill` 和 `bfill` 两种值，分别代表前向填充和后向填充，使得数据前后具有连贯性，而一般的独立性样本并不适用该函数。（前向填充是指用缺失值的前一个值替换，而后向填充是指用缺失值的后一个值替换。）

## Apply 函数

`.apply()` 函数也是一个 pandas 中常用的函数。它可以对每行 / 列（也可以对基于切片选择特定的列或行）应用一个函数并返回一个 `Series`。该函数可以是一些内置函数，如 max 函数、lambda 函数或自定义函数。

- max 函数

例如，返回 `year`, `POP`, `XRAT`, `tcgdp`, `cc`, `cg` 列的最大值（当该列有空值时，无法计算最大值，则返回空值）。

In [6]:
df[['year', 'POP', 'XRAT', 'tcgdp', 'cc', 'cg']].apply(max)

year        2009.0
POP      1323592.0
XRAT           NaN
tcgdp          NaN
cc             NaN
cg             NaN
dtype: float64

- lambda 函数，形式为 `lambda arguments: expression`

lambda 函数是一种没有函数名，但是可以被调用执行的函数。换句话说，lambda 函数不需要用 `def func_name()` 来定义函数名，而是直接定义函数体中的计算部分。关于函数体部分，`arguments` 是参数列表，可以指定一个或多个参数，就像定义普通函数一样；`expression` 是一个表达式，定义了 lambda 函数的计算逻辑，通常包括使用参数执行哪些计算，并返回计算结果。lambda 函数通常用于一次性的、简单的操作。

例如，对每一行返回数据本身，不做任何其他操作：

In [7]:
df.apply(lambda row: row, axis=1)

,country,isocode,year,POP,XRAT,Currency_Unit,ppp,tcgdp,cgdp,cgdp2,...,kg,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt
0,Afghanistan,AFG,1950,8150.368,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1951,8284.473,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1952,8425.333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1953,8573.217,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1954,8728.408,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,Zimbabwe,ZWE,2005,11639.470,2.236364e+01,Zimbabwe Dollar,39.482829,1968.205961,169.097559,184.183929,...,6.995770,9.376272,89.399427,214.739197,418.970867,418.970867,NaN,390.907086,NaN,169.097559
11396,Zimbabwe,ZWE,2006,11544.326,1.643606e+02,Zimbabwe Dollar,384.899651,2132.305773,184.705956,192.953943,...,7.648020,14.986823,81.697014,217.543648,424.754259,407.262097,NaN,377.352394,NaN,179.368685
11397,Zimbabwe,ZWE,2007,11443.187,9.675781e+03,Zimbabwe Dollar,38583.323960,2107.937100,184.208918,198.215361,...,8.387106,15.787322,84.483374,202.707080,396.486201,376.163064,NaN,345.764991,NaN,173.113448
11398,Zimbabwe,ZWE,2008,11350.000,6.715424e+09,Zimbabwe Dollar,38723.957740,1772.209867,156.141839,162.112294,...,7.685312,13.444449,85.117130,174.178806,343.159758,332.649861,NaN,302.945712,NaN,142.329054


在这个例子中，`apply` 有个参数为 `axis`，`axis = 1` 设置函数对每一行操作；`axis = 0` 设置函数对每一列操作；默认 `axis = 0`。

例：和 `.loc[]` 一起使用，进行更高级的数据切片。`.apply()` 返回对每一行做条件判断的一系列布尔值，以 `[]` 操作选择部分列。下面的选择条件为：如果 `country` 列属于特定国家，且 `POP > 40000`；如果 `country` 列不属于特定国家，且 `POP < 20000`

In [8]:
complexCondition = df.apply(
lambda row: row.POP > 40000 if row.country in ['Argentina', 'India', 'South Africa'] else row.POP < 20000,
axis=1), ['country', 'year', 'POP', 'XRAT', 'tcgdp']
complexCondition

(0        True
 1        True
 2        True
 3        True
 4        True
          ... 
 11395    True
 11396    True
 11397    True
 11398    True
 11399    True
 Length: 11400, dtype: bool,
 ['country', 'year', 'POP', 'XRAT', 'tcgdp'])

In [9]:
df.loc[complexCondition]

,country,year,POP,XRAT,tcgdp
0,Afghanistan,1950,8150.368,NaN,NaN
1,Afghanistan,1951,8284.473,NaN,NaN
2,Afghanistan,1952,8425.333,NaN,NaN
3,Afghanistan,1953,8573.217,NaN,NaN
4,Afghanistan,1954,8728.408,NaN,NaN
...,...,...,...,...,...
11395,Zimbabwe,2005,11639.470,2.236364e+01,1968.205961
11396,Zimbabwe,2006,11544.326,1.643606e+02,2132.305773
11397,Zimbabwe,2007,11443.187,9.675781e+03,2107.937100
11398,Zimbabwe,2008,11350.000,6.715424e+09,1772.209867


则返回符合条件的行和列。

## 更改 DataFrame 

更改 `DataFrame` 的部分值（行、列）在数据分析过程中十分重要。

- `.where()` 函数保留行，并用其他值替代其余行。

例：找出 POP 大于 20000 的行，保留这些行，其余部分用 `other` 替代，`other` 参数可以是任意值，比如 0 、"hello" 等。

In [10]:
df.where(df.POP > 20000, other=False)

,country,isocode,year,POP,XRAT,Currency_Unit,ppp,tcgdp,cgdp,cgdp2,...,kg,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11396,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11397,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
11398,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


- `.loc[]` 函数指定想修改的列，并赋值。

例：找出 cg 列的最大值，赋值为 1 。

In [11]:
df.loc[df.cg == max(df.cg), 'cg'] = 1

- `.apply()` 函数，根据自定义函数修改行和列。

例：将 POP 小于 10000 的修改为 1， 将 XRAT 缩小十倍。

In [12]:
def update_row(row):
  # modify POP
  row.POP = 1 if row.POP<= 10000 else row.POP

  # modify XRAT
  row.XRAT = row.XRAT / 10
  return row

df.apply(update_row, axis=1)

,country,isocode,year,POP,XRAT,Currency_Unit,ppp,tcgdp,cgdp,cgdp2,...,kg,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt
0,Afghanistan,AFG,1950,1.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1951,1.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1952,1.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1953,1.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1954,1.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,Zimbabwe,ZWE,2005,11639.470,2.236364e+00,Zimbabwe Dollar,39.482829,1968.205961,169.097559,184.183929,...,6.995770,9.376272,89.399427,214.739197,418.970867,418.970867,NaN,390.907086,NaN,169.097559
11396,Zimbabwe,ZWE,2006,11544.326,1.643606e+01,Zimbabwe Dollar,384.899651,2132.305773,184.705956,192.953943,...,7.648020,14.986823,81.697014,217.543648,424.754259,407.262097,NaN,377.352394,NaN,179.368685
11397,Zimbabwe,ZWE,2007,11443.187,9.675781e+02,Zimbabwe Dollar,38583.323960,2107.937100,184.208918,198.215361,...,8.387106,15.787322,84.483374,202.707080,396.486201,376.163064,NaN,345.764991,NaN,173.113448
11398,Zimbabwe,ZWE,2008,11350.000,6.715424e+08,Zimbabwe Dollar,38723.957740,1772.209867,156.141839,162.112294,...,7.685312,13.444449,85.117130,174.178806,343.159758,332.649861,NaN,302.945712,NaN,142.329054


- `.applymap()` 函数，修改 `DataFrame` 中的所有单独元素。

例：将 DataFrame 中的数值型元素保留两位小数。

In [13]:
df.applymap(lambda x : round(x, 2) if type(x)!=str else x)

,country,isocode,year,POP,XRAT,Currency_Unit,ppp,tcgdp,cgdp,cgdp2,...,kg,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt
0,Afghanistan,AFG,1950,8150.37,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1951,8284.47,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1952,8425.33,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1953,8573.22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1954,8728.41,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,Zimbabwe,ZWE,2005,11639.47,2.236000e+01,Zimbabwe Dollar,39.48,1968.21,169.10,184.18,...,7.00,9.38,89.40,214.74,418.97,418.97,NaN,390.91,NaN,169.10
11396,Zimbabwe,ZWE,2006,11544.33,1.643600e+02,Zimbabwe Dollar,384.90,2132.31,184.71,192.95,...,7.65,14.99,81.70,217.54,424.75,407.26,NaN,377.35,NaN,179.37
11397,Zimbabwe,ZWE,2007,11443.19,9.675780e+03,Zimbabwe Dollar,38583.32,2107.94,184.21,198.22,...,8.39,15.79,84.48,202.71,396.49,376.16,NaN,345.76,NaN,173.11
11398,Zimbabwe,ZWE,2008,11350.00,6.715424e+09,Zimbabwe Dollar,38723.96,1772.21,156.14,162.11,...,7.69,13.44,85.12,174.18,343.16,332.65,NaN,302.95,NaN,142.33


- `del` 函数删除指定列。

例：删除 `ppp` 列。

In [14]:
del df['ppp']

- `df['new_column'] = values` 增加新列。

例：增加一列 `GDP percap` 显示人均 GDP。

In [15]:
df['GDP percap'] = df['tcgdp'] / df['POP']

- `astype()` 更改变量数据类型

例：将 country 列改为字符串类型。

In [16]:
df.country = df.country.astype('str')

## 数据排序

在很多分析任务中，需要按照某个或某些指标对数据进行排序。pandas 在排序时，根据排序的对象不同可细分为 `sort_values` 和 `sort_index`, 与其字面意义相一致，分别代表了对值进行排序和对索引 index 进行排序。

- `.sort_values()` 函数，按照指定列排序。

形如 `.sort_values(by='column1', ascending = True/False)`，`ascending` 参数设置为 True 表示升序，False 为降序。例如按照 POP 列升序排列：

In [17]:
df = df.sort_values(by='POP', ascending = True)
df

,country,isocode,year,POP,XRAT,Currency_Unit,tcgdp,cgdp,cgdp2,cda2,...,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt,GDP percap
7740,Palau,PLW,1950,7.251,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7741,Palau,PLW,1951,7.448,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7742,Palau,PLW,1952,7.651,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7743,Palau,PLW,1953,7.859,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7744,Palau,PLW,1954,8.073,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158,China Version 1,CHN,2008,1317066.000,6.95,Yuan Renminbi,9.719402e+06,7379.586069,7529.397258,6747.223691,...,42.862836,68.760068,7132.315550,10938.84514,10929.28222,NaN,10937.76354,NaN,6635.526673,7.379586
2219,China Version 2,CH2,2009,1323592.000,6.83,Yuan Renminbi,1.134246e+07,8569.456643,9227.187296,8531.307128,...,39.044203,61.240342,8247.646160,NaN,NaN,NaN,12658.65025,NaN,7747.331618,8.569457
2159,China Version 1,CHN,2009,1323592.000,6.83,Yuan Renminbi,1.073933e+07,8113.773410,8296.575690,7651.109254,...,46.707842,58.579480,7782.385295,NaN,NaN,NaN,11914.32395,NaN,7314.930606,8.113773
10020,Tajikistan,TJK,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


也可以对多个列排序。例如按照 POP 列和 year 列降序排列。

In [18]:
df = df.sort_values(by=['POP','year'], ascending = False)
df

,country,isocode,year,POP,XRAT,Currency_Unit,tcgdp,cgdp,cgdp2,cda2,...,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt,GDP percap
2219,China Version 2,CH2,2009,1323592.000,6.830000,Yuan Renminbi,1.134246e+07,8569.456643,9227.187296,8531.307128,...,39.044203,61.240342,8247.646160,NaN,NaN,NaN,12658.65025,NaN,7747.331618,8.569457
2159,China Version 1,CHN,2009,1323592.000,6.830000,Yuan Renminbi,1.073933e+07,8113.773410,8296.575690,7651.109254,...,46.707842,58.579480,7782.385295,NaN,NaN,NaN,11914.32395,NaN,7314.930606,8.113773
2218,China Version 2,CH2,2008,1317066.000,6.950000,Yuan Renminbi,1.055732e+07,8015.782984,8416.480345,7551.673672,...,37.629450,69.883127,7777.945871,11929.04952,11929.04952,NaN,11938.30669,NaN,7233.090071,8.015783
2158,China Version 1,CHN,2008,1317066.000,6.950000,Yuan Renminbi,9.719402e+06,7379.586069,7529.397258,6747.223691,...,42.862836,68.760068,7132.315550,10938.84514,10929.28222,NaN,10937.76354,NaN,6635.526673,7.379586
2217,China Version 2,CH2,2007,1310584.000,7.607532,Yuan Renminbi,9.407736e+06,7178.278054,7437.401920,6803.244389,...,37.634047,69.245056,7357.713761,11290.43295,11290.43295,NaN,11288.33089,NaN,6690.086924,7.178278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7742,Palau,PLW,1952,7.651,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7741,Palau,PLW,1951,7.448,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7740,Palau,PLW,1950,7.251,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10021,Tajikistan,TJK,1951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- `.sort_index()` 函数，按照现有的 index 索引排序，index 通常为打印 `DataFrame` 时，最左侧没有任何列名的那一列。

In [19]:
df.sort_index()

,country,isocode,year,POP,XRAT,Currency_Unit,tcgdp,cgdp,cgdp2,cda2,...,ki,openk,rgdpeqa,rgdpwok,rgdpl2wok,rgdpl2pe,rgdpl2te,rgdpl2th,rgdptt,GDP percap
0,Afghanistan,AFG,1950,8150.368,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1951,8284.473,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1952,8425.333,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1953,8573.217,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1954,8728.408,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,Zimbabwe,ZWE,2005,11639.470,2.236364e+01,Zimbabwe Dollar,1968.205961,169.097559,184.183929,203.065978,...,9.376272,89.399427,214.739197,418.970867,418.970867,NaN,390.907086,NaN,169.097559,0.169098
11396,Zimbabwe,ZWE,2006,11544.326,1.643606e+02,Zimbabwe Dollar,2132.305773,184.705956,192.953943,201.489048,...,14.986823,81.697014,217.543648,424.754259,407.262097,NaN,377.352394,NaN,179.368685,0.184706
11397,Zimbabwe,ZWE,2007,11443.187,9.675781e+03,Zimbabwe Dollar,2107.937100,184.208918,198.215361,199.155404,...,15.787322,84.483374,202.707080,396.486201,376.163064,NaN,345.764991,NaN,173.113448,0.184209
11398,Zimbabwe,ZWE,2008,11350.000,6.715424e+09,Zimbabwe Dollar,1772.209867,156.141839,162.112294,170.488034,...,13.444449,85.117130,174.178806,343.159758,332.649861,NaN,302.945712,NaN,142.329054,0.156142
